# Рекомендация тарифов

**Описание проекта**  
Оператор мобильной связи «Мегалайн» выяснил: многие клиенты пользуются архивными тарифами. Они хотят построить систему, способную проанализировать поведение клиентов и предложить пользователям новый тариф: «Смарт» или «Ультра».
В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится, так как выполнялась на одном из предудущих этапах.
Постройте модель с максимально большим значением accuracy. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте `accuracy` на тестовой выборке самостоятельно.  

**Описание данных**  
  * сalls — количество звонков;
  * minutes — суммарная длительность звонков в минутах;
  * messages — количество sms-сообщений;
  * mb_used — израсходованный интернет-трафик в Мб;
  * is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).  
  
**Ход выполнения:**
  1) Откроем файл с данными и изучим его. Путь к файлу: /datasets/users_behavior.csv;  
  2) Разделим исходные данные на обучающую, валидационную и тестовую выборки;  
  3) Исследуем качество разных моделей, меняя гиперпараметры. Кратко составим выводы исследования;  
  4) Выполним проверку качества модели на тестовой выборке;  
  5) Дополнительное задание: проверить модели на вменяемость.   

## Откройте и изучите файл

Импортируем необходимые инструменты

In [1]:
import pandas as pd
import os
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.dummy import DummyClassifier

Сохраняем датафрейм в пременную и ознакомимся с данными

In [2]:
pth1 = '/datasets/users_behavior.csv'
pth2 = 'C:/Users/username/Desktop/users_behavior.csv'

if os.path.exists(pth1):
    df = pd.read_csv(pth1)
elif os.path.exists(pth2):
    df = pd.read_csv(pth2)
else:
    print('Check the PATH to data file please')
    
display(df.head())
display(df.info())

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


None

Так как тип данных в столбце `is_ultra int64`, то хорошо бы проверить на отсутствие значений, отличных от 0 и 1

In [3]:
df['is_ultra'].unique()

array([0, 1])

Вывод команды говорит нам о том, что столбец `is_ultra` не содержит других значений, кроме 0 и 1. А с учётом вывода команды `info()`, показавшей отсутствие пропусков, получается, что все данные у нас размечены.

### Выводы:  
Согласно выводу команды `info()`, все строки заполнены, пропусков нет. Заголовки корректные и содержимое столбцов не вызывает нареканий по первому взгляду, другими словами, данные подготовлены к дальнейшей работе с ними.

## Разбейте данные на выборки

Так как мы формируем три выборки из одного набора данных, то функцией `train_test_split` разбиваем его в следующих пропорциях:
обучающая (60%), валидационная (20%) и тестовая (20%).

In [4]:
# первой отделим тестовую выборку, оставшиеся 80% датасета сохраним во временную переменную
df_80, df_test = train_test_split(df, test_size=.2, random_state=12345)
# с учётом изменившихся пропорций делим свободный датасет на валидационную и обучающие выборки
df_train, df_valid = train_test_split(df_80, test_size=.25, random_state=12345)

In [5]:
# выводим на экран и проверяем содержимое выборок
display(df_train.info())
display(df_valid.info())
display(df_test.info())
# небольшая проверка на полноту
if len(df) == (len(df_train) + len(df_valid) + len(df_test)):
    print('Количество строк в датасете и выборках совпадает!')
else:
    print('Количество строк в датасете и выборках НЕ совпадает!')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1928 entries, 2656 to 510
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     1928 non-null   float64
 1   minutes   1928 non-null   float64
 2   messages  1928 non-null   float64
 3   mb_used   1928 non-null   float64
 4   is_ultra  1928 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 90.4 KB


None

<class 'pandas.core.frame.DataFrame'>
Int64Index: 643 entries, 2699 to 1806
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     643 non-null    float64
 1   minutes   643 non-null    float64
 2   messages  643 non-null    float64
 3   mb_used   643 non-null    float64
 4   is_ultra  643 non-null    int64  
dtypes: float64(4), int64(1)
memory usage: 30.1 KB


None

<class 'pandas.core.frame.DataFrame'>
Int64Index: 643 entries, 1415 to 1196
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     643 non-null    float64
 1   minutes   643 non-null    float64
 2   messages  643 non-null    float64
 3   mb_used   643 non-null    float64
 4   is_ultra  643 non-null    int64  
dtypes: float64(4), int64(1)
memory usage: 30.1 KB


None

Количество строк в датасете и выборках совпадает!


### Выводы:  
Исходный датасет был успешно разделён на три выборки.

## Исследуйте модели

Исслкдовать мы будем 3 модели: Дерево решений, Случайный лес и Логистическую регрессию.  
Можно сразу подготовить выборки, разделив их на пространство признаков и целевые признаки (`is_ultra`).

In [6]:
features_train = df_train.drop(['is_ultra'], axis=1)
target_train = df_train['is_ultra']
features_valid = df_valid.drop(['is_ultra'], axis=1)
target_valid = df_valid['is_ultra']
features_test = df_test.drop(['is_ultra'], axis=1)
target_test = df_test['is_ultra']

### Дерево решений

Модель будем запускать в цикле с подбором её основного гиперпараметра - глубины дерева.  
Как показывает наш предыдущий опыт больше 5 уровней нет смысла делать дерево, так как модель переобучается.

In [7]:
best_result = 0
best_depth = 0
best_model_tr = None

for depth in range(1, 6):
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    # обучаем модель
    model.fit(features_train, target_train)
    # предсказываем целевой признак для валидационной выборки
    predictions_valid = model.predict(features_valid)
    # проверяем accurancy для валидационной выборки
    result = accuracy_score(target_valid, predictions_valid)
    
    # сохраняем лучшие значения
    if result > best_result:
        best_depth = depth
        best_result = result
        best_model_tr = model

print('Лучший показатель глубины дерева:', best_depth)
print('Лучший показатель accurancy:', best_result)

Лучший показатель глубины дерева: 3
Лучший показатель accurancy: 0.7651632970451011


### Случайный лес

Модель также запускаем в цикле, с подбором сразу трёх гиперпараметров: количество деревьев в лесу, глубина каждого и количество элементов в листе.  
Количество деревьев будем менять от 10 до 100 с шагом 10, глубину от 1 до 5, кол-во элементов в листьях от 1 до 3.  
Обучим каждую полученную модель на обучающей выборке, проверим результат обучения на валидационной выборке и выберем из них лес с максимальной долей правильных ответов.

In [8]:
best_model_fr = None
best_est = 0
best_depth = 0
best_sample = 0
best_result = 0


for est in range(10, 101, 10):
    for depth in range(1, 6):
        for sample in range(1, 4):
            model = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth=depth, min_samples_leaf=sample)
            # обучаем модель
            model.fit(features_train, target_train)
            # проверяем результат
            result = model.score(features_valid, target_valid)
            if result > best_result:
                best_model_fr = model
                best_est = est
                best_depth = depth
                best_sample = sample
                best_result = result

print('Лучший показатель accurancy при кол-ве деревьев:', best_est)
print('Лучший показатель accurancy при глубине:', best_depth)
print('Лучший показатель accurancy при кол-ве элементов в листе:', best_sample)
print('Лучший показатель accurancy:', best_result)

Лучший показатель accurancy при кол-ве деревьев: 90
Лучший показатель accurancy при глубине: 5
Лучший показатель accurancy при кол-ве элементов в листе: 2
Лучший показатель accurancy: 0.7838258164852255


### Логистическая регрессия

Создадим модель логистической регрессии, обучаем на обучающей выборке, проверяем на валидационной.

In [9]:
model_lr = LogisticRegression()
# обучаем
model_lr.fit(features_train, target_train)
# предсказываем значение целевого признака
predictions_valid = model_lr.predict(features_valid)
# проверяем точность
accuracy = model_lr.score(features_valid, target_valid)
print('Точность модели:', accuracy)

Точность модели: 0.7262830482115086


### Выводы:  
Наилучший показатель точности продемонстрировала модель "Случайный лес" - 78.4%
Подобранные гиперпараметры: кол-во деревьев - 90, глубина - 5, кол-во элементов в листе - 2.  
Её и будем использовать для дальнейше проверки на тестовой выборке.

## Проверьте модель на тестовой выборке

In [10]:
# здесь теперь можно использовать для обучения совокупность признаков из обучающей и валидационной выборок
features_train_80 = df_80.drop(['is_ultra'], axis=1)
target_train_80 = df_80['is_ultra']

model = RandomForestClassifier(random_state=12345, n_estimators=90, max_depth=5, min_samples_leaf=2)
# обучаем модель
model.fit(features_train_80, target_train_80)
# проверяем результат
result = model.score(features_test, target_test)

print('Точность на тестовой выборке:', result)

Точность на тестовой выборке: 0.7916018662519441


### Вывод:  
Модель с оптимизированными гиперпараметрами на тестовой выборке показала точность немного лучше, чем на валидационной - 79.1%

## (бонус) Проверьте модели на адекватность

Проверка на адекватность - это сравнение основного показателя качества для модели-глупышки и специально обученной.  
В качестве глупышки воспользуемся DummyClassifier, долю правильных ответов посчитаем с параметром `strategy='most_frequent'`

In [11]:
# вызываем модель
dummy = DummyClassifier(strategy='most_frequent')
# обучаем
dummy.fit(features_train, target_train)
# предсказываем
dummy.predict(features_test)
# смотрим точность
dummy.score(features_test, target_test)

0.6951788491446346

### Вывод:  
Полученная нами модель "Случайный лес" успешно проходит проверку на адекватность, так как доля её верных ответов выше, чем при случайном угадывании: 79% против 69.5%

## Общий вывод по проведённому исследованию:  
  1) В исследовании участвовали три различные алгоритма классификации: дерево решений, случайный лес и логистическая регрессия.  
  2) Для проведения обучения, проверки моделей и тестирования лучшей модели исходный датафрейм был разделён на три выборки: обучающую (60%), валидационную (20%) и тестовую (20%).  
  3) По итогам исследования можно сделать заключение о том, что для получения рекомендации тарифных планов наиболее подходящей моделью является модель, построенная на случайном лесе с количеством деревьев 90, глубиной каждого отдельного дерева 5 и количеством образцов в листе 2 - доля правильных ответов в тестовой выборке составила 79.1%  
  4) При проверке модели на адекватность сделан вывод, что она показывает намного лучший результат, чем модель с примитивным прогнозом.